## Py City Schools Homework Assignment 
    Lynne A. Freilich                 April 1, 2022

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load 
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset  
SDC = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Create a working copy of the data 
df = SDC

# Create a summary DF to count and/or average district data 
DistrictSum = pd.DataFrame( {'Total School': [len(pd.unique(df['school_name']))],
                            'Total Student': [len(pd.unique(df['Student ID']))],
                            'Total Budget': [school_data['budget'].sum()],
                            'Average Math Score': [df['math_score'].mean()],
                            'Average Reading Score': [df['reading_score'].mean()],
                            '% Passing Math': [df[df['math_score'] > 69]['math_score'].count() / df['math_score'].count() * 100],
                            '% Passing Reading': [df[df['reading_score'] > 69]['reading_score'].count() / df['reading_score'].count() * 100],
                            '% Overall Passing': [df.loc[(df["reading_score"] >= 70) & ( df["math_score"] >= 70)]['reading_score'].count() /
                                               df['reading_score'].count() * 100]
                            })

# Clean up DistrictSum for display with some mapping
DistrictSum['Total Student'] = DistrictSum['Total Student'].map("{:,.0f}".format)
DistrictSum['Total Budget'] = DistrictSum['Total Budget'].map("${:,.2f}".format)
DistrictSum['% Passing Math'] = (DistrictSum['% Passing Math']).map("{:.2f}%".format)
DistrictSum['% Passing Reading'] = (DistrictSum['% Passing Reading']).map("{:.2f}%".format)
DistrictSum['% Overall Passing'] = (DistrictSum['% Overall Passing']).map("{:.2f}%".format)

DistrictSum

,Total School,Total Student,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
# Rename school_name to School and type to School Type
SDC.rename(columns={'school_name':'School'}, inplace=True) 
SDC.rename(columns={'type':'School Type'}, inplace=True) 

# Get average math scores by school
average_math = SDC.groupby('School')['math_score'].mean()
average_reading = SDC.groupby('School')['reading_score'].mean()

# Get passing scores counts by School
AMS = SDC[SDC['math_score'] > 69].groupby('School')['math_score'].count()
ARS = SDC[SDC['reading_score'] > 69].groupby('School')['reading_score'].count()

# Get Passing count for total population
passing_count = SDC.loc[(df["reading_score"] >= 70) & ( SDC["math_score"] >= 70)] \
                        .groupby(['School'])['math_score'].count()
                      
# Get student counts by school                    
students_by_school = SDC.groupby('School').count()

# Reset series to be used in calculations
passing_count = passing_count.reset_index()
AMS = AMS.reset_index()
ARS = ARS.reset_index()

In [4]:
# Variable to hold new Column Names
NewNames = {'budget':'Total School Budget',
            'math_score':'Average Math Score',
            'student_name': 'Total Students',
            'reading_score':'Average Reading Score'}
# Variable to hold new Column Values
Aggregates = {'student_name':'count',
              'budget':'max',
              'reading_score':'mean',
              'math_score':'mean'}

# Rename additional Columns and calcuate their Values using variables NewNames and Aggregates
GS = pd.DataFrame(SDC.groupby(['School', 'School Type']).agg(Aggregates).rename(columns=NewNames)).reset_index()

# Add some calulated Columns to our Data Set
GS['Per Student Budget'] = GS['Total School Budget']/ GS['Total Students']
GS['% Overall Passing'] = passing_count['math_score'] / GS['Total Students']
GS['% Passing Math'] = AMS['math_score']/ GS['Total Students']
GS['% Passing Reading'] = ARS['reading_score']/ GS['Total Students']

# Set index to school
GS = GS.set_index('School')

In [5]:
# Create New DataFrame with the columns in desired order
School_Summary_df = pd.DataFrame(GS[['School Type','Total Students','Total School Budget',
                                 'Per Student Budget','Average Math Score',
                                 'Average Reading Score','% Passing Math','% Passing Reading',
                                 '% Overall Passing']])

# Correct some data types for use in later caculations 
School_Summary_df['% Overall Passing'] = School_Summary_df['% Overall Passing'].astype('float64') 
School_Summary_df['% Passing Math'] = School_Summary_df['% Passing Math'].astype('float64') 
School_Summary_df['% Passing Reading'] = School_Summary_df['% Passing Reading'].astype('float64')

# Copy of data frame after casting percentage fields as numbers
# Before formatting for use in final three sections 
Scores_df = pd.DataFrame(School_Summary_df).copy()

In [6]:
# Clean up School_Summary_df for display with some mapping
School_Summary_df['Total School Budget'] = School_Summary_df['Total School Budget'].map("${:,.2f}".format)
School_Summary_df['Per Student Budget'] = School_Summary_df['Per Student Budget'].map("${:,.2f}".format)
School_Summary_df['% Passing Math'] = (School_Summary_df['% Passing Math']*100).map("{:.1f}%".format)
School_Summary_df['% Passing Reading'] = (School_Summary_df['% Passing Reading']*100).map("{:.1f}%".format)
School_Summary_df['% Overall Passing'] = (School_Summary_df['% Overall Passing']*100).map("{:.1f}%".format)

School_Summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.7%,81.9%,54.6%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.1%,97.0%,91.3%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,66.0%,80.7%,53.2%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.3%,79.3%,54.3%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.4%,97.1%,90.6%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.8%,80.9%,53.5%
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.5%,96.3%,89.2%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.7%,81.3%,53.5%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.1%,81.2%,53.5%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [7]:
# Sort by percent overall passing from the best descending and diaply top five.
School_Summary_df.sort_values(by='% Overall Passing', ascending=False).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.1%,97.0%,91.3%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.3%,97.3%,90.9%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.4%,97.1%,90.6%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.9%,96.5%,90.6%
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.6%,95.9%,90.5%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [8]:
# Sort by percent overall passing from the worst and display top five.
School_Summary_df.sort_values(by='% Overall Passing', ascending=True).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.4%,80.2%,53.0%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,66.0%,80.7%,53.2%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.8%,80.9%,53.5%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.7%,81.3%,53.5%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.1%,81.2%,53.5%


## Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [9]:
# Create working copy of df
df = student_data

# Create series for 9th Grade
math_by_grade = df[df['grade'] == '9th'].groupby('school_name').agg({'math_score': 'mean'}).reset_index()
math_by_grade = math_by_grade.rename(columns = {'math_score': '9th'})

# Create series for 10th Grade and Merge to 9th grade
math_by_grade = pd.merge(math_by_grade,
                         df[df['grade'] == '10th'].groupby('school_name').agg({'math_score': 'mean'}).reset_index(), 
                         how="inner", on=['school_name', "school_name"], sort=True)
math_by_grade = math_by_grade.rename(columns = {'math_score': '10th'})

# Create series for 11th Grade and Merge to 9th grade and 10th Grades
math_by_grade = pd.merge(math_by_grade,
                         df[df['grade'] == '11th'].groupby('school_name').agg({'math_score': 'mean'}).reset_index(),
                         how="inner", on=["school_name", "school_name"], sort=True)
math_by_grade = math_by_grade.rename(columns = {'math_score': '11th'})

# Create series for 12th Grade and Merge to 9th, 10th and 11th Grades
math_by_grade = pd.merge(math_by_grade,
                         df[df['grade'] == '12th'].groupby('school_name').agg({'math_score': 'mean'}).reset_index(),
                         how="inner", on=["school_name", "school_name"], sort=True)
math_by_grade = math_by_grade.rename(columns = {'math_score': '12th', 'school_name': ''})

math_by_grade.style.hide_index()

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [10]:
# Refresh working copy of df
df = student_data

# Create series for 9th Grade
reading_by_grade = df[df['grade'] == '9th'].groupby('school_name').agg({'reading_score': 'mean'}).reset_index()
reading_by_grade = reading_by_grade.rename(columns = {'reading_score': '9th'})


# Create series for 10th Grade and Merge to 9th grade
reading_by_grade = pd.merge(reading_by_grade,
                         df[df['grade'] == '10th'].groupby('school_name').agg({'reading_score': 'mean'}).reset_index(), 
                         how="inner", on=["school_name", "school_name"], sort=True)
reading_by_grade = reading_by_grade.rename(columns = {'reading_score': '10th'})


# Create series for 11th Grade and Merge to 9th grade and 10th Grades
reading_by_grade = pd.merge(reading_by_grade,
                         df[df['grade'] == '11th'].groupby('school_name').agg({'reading_score': 'mean'}).reset_index(),
                         how="inner", on=["school_name", "school_name"], sort=True)
reading_by_grade = reading_by_grade.rename(columns = {'reading_score': '11th'})


# Create series for 12th Grade and Merge to 9th, 10th and 11th Grades
reading_by_grade = pd.merge(reading_by_grade,
                         df[df['grade'] == '12th'].groupby('school_name').agg({'reading_score': 'mean'}).reset_index(),
                         how="inner", on=["school_name", "school_name"], sort=True)
reading_by_grade = reading_by_grade.rename(columns = {'reading_score': '12th', 'school_name': ''})

reading_by_grade.style.hide_index()

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [11]:
# Create working copy of df with explicit copy as some data will be replaced
Scores = Scores_df.copy()

# Fix percentages for display
Scores['% Passing Math'] = Scores['% Passing Math'] * 100
Scores['% Passing Reading'] = Scores['% Passing Reading'] * 100
Scores['% Overall Passing'] = Scores['% Overall Passing'] * 100

# Create bins to hold per student budget information
bins = [0, 585, 630, 645, 680]

# Create labels for the bins holding per student budgets
binLabels = ["< \$585", "\$586 to \$630",
             "\$631 to \$645", "\$646 to \$680"]

# Cut per student budget data to bins
Scores['Spending Ranges (Per Student)'] =  pd.cut(Scores['Per Student Budget'], bins, labels=binLabels)

# group binned data 
Scores_group = Scores.groupby('Spending Ranges (Per Student)')

# Build df for output 
Scores_group[['Average Math Score','Average Reading Score',
              '% Passing Math','% Passing Reading','% Overall Passing']].mean()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
< \$585,83.455399,83.933814,93.460096,96.610877,90.369459
\$586 to \$630,81.899826,83.155286,87.133538,92.718205,81.418596
\$631 to \$645,78.518855,81.624473,73.484209,84.391793,62.857656
\$646 to \$680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

* Perform the same operations as above, based on school size.

In [12]:
# Create working copy of df with explicit copy as some data will be replaced
Scores = Scores_df.copy()

# Fix percentages for display
Scores['% Passing Math'] = Scores['% Passing Math'] * 100
Scores['% Passing Reading'] = Scores['% Passing Reading'] * 100
Scores['% Overall Passing'] = Scores['% Overall Passing'] * 100

# Create bins to hold school size information
bins = [0, 1000, 2000, 5000]

# Create labels for the bins holding school size information
binLabels = ['Small (<1000)', 'Medium (1000-2000)',
             'Large (2000-5000']

# Cut per school size data to bins
Scores['School Size'] =  pd.cut(Scores['Total Students'], bins, labels=binLabels)

# group binned data 
Scores_group = Scores.groupby('School Size')

# Build df for output 
Scores_group[['Average Math Score','Average Reading Score',
              '% Passing Math', '% Passing Reading','% Overall Passing']].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000,77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [13]:
# Create working copy of df with explicit copy as some data will be replaced
Scores = Scores_df.copy()

# Fix percentages for display
Scores['% Passing Math'] = Scores['% Passing Math'] * 100
Scores['% Passing Reading'] = Scores['% Passing Reading'] * 100
Scores['% Overall Passing'] = Scores['% Overall Passing'] * 100

# Convert School type to numeric value for binning
Scores.loc[Scores['School Type'] == 'Charter', 'School Type']= 1
Scores.loc[Scores['School Type'] == 'District', 'School Type']= 2

# Create bins to hold school type information 
bins = [0, 1, 2]

# Create labels for the school types 
binLabels = ['Charter', 'District']

# Cut per school type data to bins
Scores['School Type'] =  pd.cut(Scores['School Type'], bins, labels=binLabels)

# group binned data 
Scores_group = Scores.groupby('School Type')

# Build df for output 
Scores_group[['Average Math Score','Average Reading Score',
              '% Passing Math','% Passing Reading','% Overall Passing']].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
